**Model Health Dashboard**


This dashboard reports statistics about the health of the collection of Drawdown solution models.

In [13]:
import importlib
import os.path

import bqplot
import IPython.display
import ipywidgets
import pandas as pd

import solution.factory

all_solutions = pd.read_csv(os.path.join('data', 'overview', 'solutions.csv'), index_col=False,
                            skipinitialspace=True, header=0, skip_blank_lines=True, comment='#')
total_solutions = len(all_solutions.index)
py_solutions_scenarios = solution.factory.all_solutions_scenarios()

num_py_solutions = len(list(py_solutions_scenarios.keys()))
num_xl_solutions = total_solutions - num_py_solutions

pds_adoption_basis_counts = {
    'Linear': 0,
    'Existing Adoption Prognostications': 0,
    'Bass Diffusion S-Curve': 0,
    'Logistic S-Curve': 0,
    'Fully Customized PDS': 0,
    'Customized S-Curve Adoption': 0,
}
ref_adoption_basis_counts = {
    'Default': 0,
    'Custom': 0,
}
scenarios_per_solution = []
for name in py_solutions_scenarios.keys():
    m = importlib.import_module('solution.'+name)
    scenarios_per_solution.append(len(m.scenarios))
    for scenario in m.scenarios.values():
        pds_adoption_basis_counts[scenario.soln_pds_adoption_basis] += 1
        ref_basis = scenario.soln_ref_adoption_basis
        ref_basis = ref_basis if ref_basis is not None else 'Default'
        ref_adoption_basis_counts[ref_basis] += 1

soln_count_chart = bqplot.Pie(sizes=[num_xl_solutions, num_py_solutions], labels=['Excel only', 'Python & Excel'],
                              display_values=True, values_format='d', display_labels='inside', radius=110)
soln_count_fig = bqplot.Figure(marks=[soln_count_chart], title='Solution implementation')
soln_count_fig.layout.width = '300px'
soln_count_fig.fig_margin = {'left':1, 'right':1, 'top':1, 'bottom':1}
pds_adoption_chart = bqplot.Pie(sizes=list(pds_adoption_basis_counts.values()),
                                labels=list(pds_adoption_basis_counts.keys()),
                                display_labels='outside', radius=140,
                                sort=False, display_values=True, values_format='d')
pds_adoption_fig = bqplot.Figure(marks=[pds_adoption_chart], title='PDS Adoption Basis')
pds_adoption_fig.layout.width = '800px'
pds_adoption_fig.fig_margin = {'left':1, 'right':1, 'top':1, 'bottom':1}

ref_adoption_chart = bqplot.Pie(sizes=list(ref_adoption_basis_counts.values()),
                                labels=list(ref_adoption_basis_counts.keys()),
                                display_labels='inside', radius=110,
                                display_values=True, values_format='d')
ref_adoption_fig = bqplot.Figure(marks=[ref_adoption_chart], title='REF Adoption Basis')
ref_adoption_fig.layout.width = '300px'
ref_adoption_fig.fig_margin = {'left':1, 'right':1, 'top':1, 'bottom':1}


# ------------------- Scenarios per solution -------------------


y_scale = bqplot.LinearScale()
y_axis = bqplot.Axis(scale=y_scale, orientation='vertical', label='solutions')
x_scale = bqplot.LinearScale()
x_axis = bqplot.Axis(scale=x_scale, tick_format='d', label='num scenarios')
hist = bqplot.Hist(sample=scenarios_per_solution, scales={'sample': x_scale, 'count': y_scale})
scenarios_per_solution_chart = bqplot.Figure(marks=[hist], axes=[x_axis, y_axis], padding_y=0,
                                             title="Num scenarios per solution")
scenarios_per_solution_chart.layout.width = '50%'
scenarios_per_solution_chart.layout.height = '300px'


# ------------------- Regional Data -------------------


surveydata = pd.read_csv(os.path.join('data', 'health', 'survey.csv'), index_col=False,
                            skipinitialspace=True, header=0, skip_blank_lines=True, comment='#')
num_scenarios = len(surveydata.index)
regional_nonzero_adoption = surveydata.loc[surveydata['RegionalFractionAdoption'] != 0.0]
regional_nonzero_tam = surveydata.loc[surveydata['RegionalFractionTAM'] != 0.0]

nonzero_count = regional_nonzero_tam.shape[0]
zero_count = num_scenarios - nonzero_count
nonzero_chart = bqplot.Pie(sizes=[nonzero_count, zero_count], labels=['YES', 'NO'], colors=['Green', 'Red'],
                           display_values=True, values_format='d', display_labels='inside', radius=110)
tam_regional_nonzero_fig = bqplot.Figure(marks=[nonzero_chart], title='Has Regional TAM Data?')
tam_regional_nonzero_fig.layout.width = '320px'
tam_regional_nonzero_fig.fig_margin = {'left':1, 'right':1, 'top':1, 'bottom':1}

y_scale = bqplot.LinearScale()
y_axis = bqplot.Axis(scale=y_scale, orientation='vertical', label='# scenarios')
x_scale = bqplot.LinearScale()
x_axis = bqplot.Axis(scale=x_scale, tick_format='.1f', label='percentage')
hist_data = regional_nonzero_tam['RegionalFractionTAM'] * 100.0
hist = bqplot.Hist(sample=hist_data, scales={'sample': x_scale, 'count': y_scale})
hist.bins = 50
tam_regional_chart = bqplot.Figure(marks=[hist], axes=[x_axis, y_axis], padding_y=0,
                                  title="Regional TAM as a % of World")
tam_regional_chart.layout.width = '100%'

nonzero_count = regional_nonzero_adoption.shape[0]
zero_count = num_scenarios - nonzero_count
nonzero_chart = bqplot.Pie(sizes=[nonzero_count, zero_count], labels=['YES', 'NO'], colors=['Green', 'Red'],
                           display_values=True, values_format='d', display_labels='inside', radius=110)
ad_regional_nonzero_fig = bqplot.Figure(marks=[nonzero_chart], title='Has Regional Adoption Data?')
ad_regional_nonzero_fig.layout.width = '320px'
ad_regional_nonzero_fig.fig_margin = {'left':1, 'right':1, 'top':1, 'bottom':1}

y_scale = bqplot.LinearScale()
y_axis = bqplot.Axis(scale=y_scale, orientation='vertical', label='# scenarios')
x_scale = bqplot.LinearScale()
x_axis = bqplot.Axis(scale=x_scale, tick_format='.1f', label='percentage')
hist_data = regional_nonzero_adoption['RegionalFractionAdoption'] * 100.0
hist = bqplot.Hist(sample=hist_data, scales={'sample': x_scale, 'count': y_scale})
hist.bins = 100
ad_regional_chart = bqplot.Figure(marks=[hist], axes=[x_axis, y_axis], padding_y=0,
                                  title="Regional Adoption as a % of World")
ad_regional_chart.layout.width = '100%'


y_scale = bqplot.LinearScale()
y_axis = bqplot.Axis(scale=y_scale, orientation='vertical', label='# scenarios')
x_scale = bqplot.LinearScale()
x_axis = bqplot.Axis(scale=x_scale, tick_format='.1f', label=u'R\u00B2 value')
hist_data = surveydata['Rvalue'].dropna() ** 2
hist = bqplot.Hist(sample=hist_data, scales={'sample': x_scale, 'count': y_scale})
hist.bins = 200
r2_value_chart = bqplot.Figure(marks=[hist], axes=[x_axis, y_axis], padding_y=0,
                                  title="Linearity of PDS Adoption")
r2_value_chart.layout.width = '100%'

ipywidgets.VBox([
    ipywidgets.HBox([soln_count_fig, pds_adoption_fig, ref_adoption_fig]),
    scenarios_per_solution_chart,
    ipywidgets.HBox([tam_regional_nonzero_fig, tam_regional_chart]),
    ipywidgets.HBox([ad_regional_nonzero_fig, ad_regional_chart]),
    r2_value_chart,
])